
# Viết chương trình sử dụng DTW để nhận dạng khẩu lệnh đơn lẻ

Notebook này chỉ mô tả cách nhận diện một khẩu lệnh duy nhất, các khẩu lệnh còn lại đều được thực hiện tương tự. Chương trình đầy đủ nằm tại [dtw.py]()


## Đọc, xử lý dữ liệu, tách đặc trưng


In [1]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as dsp
import numpy as np

Khẩu lệnh cần nhận dạng là  ***lên***, biến `word` có thể thay đổi thành `a`, `b`, `len`, `xuong`, `trai`, `phai`, `ban` hoặc `nhay` để huấn luyện đơn lẻ từng khẩu lệnh tương ứng 

In [2]:
cmd = 'len'

In [3]:
dir = !pwd
path = dir.s + '/wav'

p_train = path + '/train/' + cmd

fst = '/1.wav'
scd = '/2.wav'
thd = '/3.wav'

In [4]:
p1 = p_train + fst
p2 = p_train + scd
p3 = p_train + thd

Nghe qua các khẩu lệnh được sử dụng để xây dựng template

In [5]:
dsp.Audio(p1)

In [6]:
dsp.Audio(p2)

In [7]:
dsp.Audio(p3)

Đọc âm thanh bằng thư viện `librosa`

In [8]:
a1, _ = librosa.load(p1)
a2, _ = librosa.load(p2)
a3, _ = librosa.load(p3)

Trích xuất đặc trưng MFCC, delta, deltadelta của từng đoạn âm thanh

In [9]:
mfcc1   = librosa.feature.mfcc(y=a1)
delta1  = librosa.feature.delta(mfcc1)
delta21 = librosa.feature.delta(mfcc1, order=2)

mfcc2   = librosa.feature.mfcc(y=a2)
delta2  = librosa.feature.delta(mfcc2)
delta22 = librosa.feature.delta(mfcc2, order=2)

mfcc3   = librosa.feature.mfcc(y=a3)
delta3  = librosa.feature.delta(mfcc3)
delta23 = librosa.feature.delta(mfcc3, order=2)

Kết hợp cả 3 đặc trưng thu được thành một vectơ đặc trưng duy nhất. Vectơ này sẽ được coi là template của một đoạn âm thanh. Số đặc trưng MFCC được lựa chọn mặc định là 20, do đó số chiều của vectơ đặc trưng sẽ là (60, x)

In [10]:
ft1 = np.concatenate((mfcc1, delta1, delta21))
ft2 = np.concatenate((mfcc2, delta2, delta22))
ft3 = np.concatenate((mfcc3, delta3, delta23))

In [11]:
ft1.shape, ft2.shape, ft3.shape

((60, 22), (60, 18), (60, 11))

Tiếp theo cần phải chuẩn hóa các vectơ đặc trưng này. Đồng thời ta sẽ chọn một vectơ ngẫu nhiên làm template chuẩn để dóng với các template khác

In [12]:
def normalize(feature):
    normalized = np.full_like(feature, 0)
    for i in range(feature.shape[1]):
        normalized[:,i] = feature[:,i] - np.mean(feature[:,i]) # đưa trung bình về 0
        normalized[:,i] = normalized[:,i] / np.max(np.abs(normalized[:,i])) # đưa khoảng giá trị về [-1, 1]
    return normalized

In [13]:
template0 = normalize(ft1) # Lựa chọn ft1 làm template chuẩn
template1 = normalize(ft2)
template2 = normalize(ft3)

Do DTW thuần được sử dụng cho các chuỗi số (tức vectơ), vì vậy khi sử dụng với các template (là một ma trận) ta sẽ cần thêm một thước đo tính chi phí để dóng 2 vectơ. Thước đo mặc định được sử dụng là khoảng cách Euclid  

In [14]:
metric = 'cosine'

In [15]:
metric = 'euclidean'

Thực hiện dóng hàng template1 và template2 theo template 0 (template chuẩn)

In [16]:
cost01, align01 = librosa.sequence.dtw(template0, template1, metric=metric)
cost02, align02 = librosa.sequence.dtw(template0, template2, metric=metric)

Hàm `librosa.sequence.dtw` trả về ma trận chi phí là bảng quy hoạch động thu được khi dóng hàng 2 chuỗi, và ma trận kích cỡ (x, 2) chứa các cặp chỉ số tương ứng giữa 2 chuỗi.

In [17]:
print(template0.shape, template1.shape, cost01.shape, sep='\n')

(60, 22)
(60, 18)
(22, 18)



## Xây dựng template trung bình


In [18]:
count = np.ones(len(template0[0])) # count[i] đếm số vectơ được dóng vào vị trí i của template chuẩn
summ = template0.copy() # tổng của 3 template sau khi dóng

Gộp template1 và template2 vào template chuẩn

In [19]:
for t, q in align01:
    count[t] += 1
    summ[:,t] += template1[:,q]

In [20]:
for t, q in align02:
    count[t] += 1
    summ[:,t] += template2[:,q]

Lấy trung bình, ta thu được template trung bình có kích thước giống kích thước template chuẩn

In [21]:
average_template = summ / count
average_template.shape == template0.shape

True


## Nhận diện khẩu lệnh bằng cách so khớp với template trung bình*


*Do mới chỉ xây dựng template trung bình cho khẩu lệnh ***lên***, hiện tại ta chưa nhận diện một khẩu lệnh bất kỳ (chương trình nhận diện đầy đủ khẩu lệnh có tại [dtw.py]()). Tuy nhiên ta vẫn có thể kiểm tra bằng cách tính chi phí dóng hàng giữa template chuẩn và các template test. Nếu chi phí giữa template chuẩn và template của khẩu lệnh "lên" là nhỏ nhất, ta có thể tin rằng mọi thứ đang đi đúng hướng.

### Đọc và nghe thử khẩu lệnh, bắt đầu bằng khẩu lệnh ***a***

In [22]:
cmd = ['a', 'b', 'lên', 'xuống', 'trái', 'phải', 'bắn', 'nhảy']
p_test = path + '/test'

In [23]:
a_test, _ = librosa.load(p_test + '/a1.wav')

In [24]:
dsp.Audio(p_test + '/a1.wav')

### Trích xuất đặc trưng, xây dựng template và tìm chi phí dóng hàng với template trung bình

In [25]:
mfcc_test     = librosa.feature.mfcc(y=a_test)
delta_test    = librosa.feature.delta(mfcc_test)
delta2_test   = librosa.feature.delta(mfcc_test, order=2)
template_test = normalize(np.concatenate((mfcc_test, delta_test, delta2_test)))

Tính "khoảng cách" giữa template của khẩu lệnh và template trung bình. Giá trị càng nhỏ thì 2 template càng giống nhau

In [26]:
cost_avg_test, _ = librosa.sequence.dtw(average_template, template_test)
cost_avg_test[-1,-1]

18.728895745890867

### Tương tự với tất cả khẩu lệnh còn lại, lần này nhanh hơn 1 chút

In [27]:
cost = [cost_avg_test[-1,-1]]

In [28]:
def get_template(path):
    audio = librosa.load(path)[0]
    mfcc = librosa.feature.mfcc(y=audio)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    return normalize(np.concatenate((mfcc, delta, delta2)))

In [29]:
c = librosa.sequence.dtw(get_template(p_test+'/b1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

13.749893968445516

In [30]:
c = librosa.sequence.dtw(get_template(p_test+'/len1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

16.83910189426994

In [31]:
c = librosa.sequence.dtw(get_template(p_test+'/xuong1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

18.780568769149127

In [32]:
c = librosa.sequence.dtw(get_template(p_test+'/trai1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

17.19753038856066

In [33]:
c = librosa.sequence.dtw(get_template(p_test+'/phai1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

14.008355887619981

In [34]:
c = librosa.sequence.dtw(get_template(p_test+'/ban1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

13.364039718936933

In [35]:
c = librosa.sequence.dtw(get_template(p_test+'/nhay1.wav'), average_template, metric=metric)[0][-1,-1]
cost.append(c)
c

15.783933320490373


## Kết quả


In [36]:
cmd[np.argmin(cost)]

'bắn'

Như vậy template trung bình mà ta đã tạo có khả năng chỉ ra được khẩu lệnh tương ứng với nó (cụ thể là khẩu lệnh "lên")

Áp dụng tương tự với các khẩu lệnh còn lại, ta sẽ có được nhiều template trung bình của nhiều khẩu lệnh. Lúc ấy thay vì so sánh nhiều khẩu lệnh với một template như trong notebook này, ta sẽ so sánh template của khẩu lệnh cần nhận dạng với các template trung bình và đưa ra khẩu lệnh có chi phí dóng hàng là nhỏ nhất. Do template trung bình có thể phát hiện ra khẩu lệnh tương ứng với nó, ta có thể tin rằng việc so sánh này cũng sẽ đưa ra kết quả chính xác. Cài đặt đầy đủ được tìm thấy tại [dtw.py]()